## Задание 10
Распознавание нарисованной цифры

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [2]:
num_classes = 10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# преобразование векторных классов в бинарные матрицы
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('Размерность x_train:', x_train.shape)
print(x_train.shape[0], 'Размер train')
print(x_test.shape[0], 'Размер test')

Размерность x_train: (60000, 28, 28, 1)
60000 Размер train
10000 Размер test


In [3]:
batch_size = 128
epochs = 8

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [4]:
hist = model.fit(x_train, y_train, batch_size = batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
print("Модель успешно обучена")

model.save('mnist.h5')
print("Модель сохранена как mnist.h5")

Epoch 1/8
469/469 [==============================] - 47s 100ms/step - loss: 2.2972 - accuracy: 0.1289 - val_loss: 2.2539 - val_accuracy: 0.3673

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Потери на тесте:', score[0])
print('Точность на тесте:', score[1])

Потери на тесте: 1.1184403896331787
Точность на тесте: 0.8008999824523926


In [7]:
from PIL import Image
from keras.models import load_model
from enum import Enum
import pygame
import numpy as np
import random


WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

class Button(Enum):
    Left = 1
    Right = 3

model = load_model('mnist.h5')

def predict_digit(img):

    # изменение рзмера изобржений на 28x28
    img = img.resize((28,28))

    # конвертируем rgb в grayscale
    img = img.convert('L')
    img = np.array(img)

    img = img.reshape(1,28,28,1)
    img = img/255.0

    res = model.predict([img])[0]
    print(res)
    return np.argmax(res), max(res)


def save_picture_and_predict(screen):
    path = 'digit.png'
    pygame.image.save(screen, path)
    img = Image.open(path)
    predict_digit(img)


screen = pygame.display.set_mode((280, 280))
line_start = None

while True:
    for i in pygame.event.get():
        if i.type == pygame.QUIT:
            pygame.quit()
            break
        if i.type == pygame.MOUSEBUTTONDOWN:
            if i.button == Button.Right.value:
                save_picture_and_predict(screen)
        elif i.type == pygame.KEYDOWN and i.key == pygame.K_SPACE:
            screen.fill(BLACK)
    pos = pygame.mouse.get_pos()
    pressed = pygame.mouse.get_pressed()
    if pressed[0]:
        pygame.draw.circle(screen,
                           WHITE,
                           (pos[0] + random.randrange(2), pos[1] + random.randrange(2)), random.randrange(1, 5))
    pygame.display.update()

[0.07781478 0.10678975 0.1470066  0.11603636 0.06256823 0.08886776
 0.12040504 0.0719566  0.12771477 0.08084016]


KeyboardInterrupt: 